In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE':128,
    'EPOCHS':50,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':8,
    'SEED':1203,
    'N_FOLDS':5
}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [4]:
df = pd.read_csv('/kaggle/input/dacon-car-crash/train.csv')

df.loc[df['sample_id'].isin(['TRAIN_2236', 'TRAIN_2596']), 'label'] = 0
df.loc[df['sample_id'].isin(['TRAIN_0061', 'TRAIN_0107', 'TRAIN_0123', 'TRAIN_0294',
    'TRAIN_0800', 'TRAIN_1280', 'TRAIN_1590', 'TRAIN_2302', 'TRAIN_2548']), 'label'] = 1
df.loc[df['sample_id'].isin(['TRAIN_0056', 'TRAIN_0129', 'TRAIN_0149', 'TRAIN_0242',
    'TRAIN_0263', 'TRAIN_0728', 'TRAIN_0861', 'TRAIN_0889', 'TRAIN_0896', 'TRAIN_0920', 'TRAIN_1098',
    'TRAIN_1169', 'TRAIN_1251', 'TRAIN_1605', 'TRAIN_1654', 'TRAIN_1656', 'TRAIN_1698', 'TRAIN_1795',
    'TRAIN_1839', 'TRAIN_1955', 'TRAIN_2249', 'TRAIN_2388', 'TRAIN_2647']), 'label'] = 3
df.loc[df['sample_id'].isin(['TRAIN_0221', 'TRAIN_0856', 'TRAIN_1081', 'TRAIN_1263',
    'TRAIN_1488', 'TRAIN_1492', 'TRAIN_1874', 'TRAIN_2166', 'TRAIN_2555', 'TRAIN_2595', 'TRAIN_2622']),
    'label'] = 4
df.loc[df['sample_id'].isin(['TRAIN_0017', 'TRAIN_0225', 'TRAIN_0306', 'TRAIN_1193',
    'TRAIN_1771', 'TRAIN_1848', 'TRAIN_2140', 'TRAIN_2298', 'TRAIN_2532', 'TRAIN_2570']), 'label'] = 5
df.loc[df['sample_id'].isin(['TRAIN_0809']), 'label'] = 6
df.loc[df['sample_id'].isin(['TRAIN_0020', 'TRAIN_0507', 'TRAIN_0617', 'TRAIN_1023',
    'TRAIN_1420', 'TRAIN_1531', 'TRAIN_2033', 'TRAIN_2063']), 'label'] = 7
df.loc[df['sample_id'].isin(['TRAIN_0332', 'TRAIN_0674', 'TRAIN_0720', 'TRAIN_0917',
    'TRAIN_1287', 'TRAIN_1699', 'TRAIN_1923', 'TRAIN_1949', 'TRAIN_2239', 'TRAIN_2491', 'TRAIN_2534', 'TRAIN_2615']), 'label'] = 9
df.loc[df['sample_id'].isin(['TRAIN_0877', 'TRAIN_1728', 'TRAIN_2328', 'TRAIN_2685']), 'label'] = 10
df.loc[df['sample_id'].isin(['TRAIN_0341', 'TRAIN_1041', 'TRAIN_1581', 'TRAIN_1727', 'TRAIN_2607']), 'label'] = 11
df.loc[df['sample_id'].isin(['TRAIN_2571']), 'label'] = 12

# 삭제할 sample_id 리스트
del_list = ['TRAIN_0048', 'TRAIN_0234', 'TRAIN_0238', 'TRAIN_0325', 'TRAIN_0528', 'TRAIN_0554', 'TRAIN_0668',
                  'TRAIN_0705', 'TRAIN_0875', 'TRAIN_1082', 'TRAIN_1151', 'TRAIN_1337', 'TRAIN_1362', 'TRAIN_1506',
                  'TRAIN_1674', 'TRAIN_1681', 'TRAIN_1753', 'TRAIN_1838', 'TRAIN_2191', 'TRAIN_2356', 'TRAIN_2360',
                  'TRAIN_2428', 'TRAIN_2451', 'TRAIN_2486', 'TRAIN_2558', 'TRAIN_2658']

# sample_id가 삭제할 리스트에 포함되지 않는 경우만 추출하여 새로운 데이터프레임 생성
df = df[~df['sample_id'].isin(del_list)]
df = df[df['label'] != 0] # 0인라벨 제거
df['label'] = df['label'].apply(lambda x: 0 if x < 7 else 1)
df['video_path'] = df['video_path'].str.replace('./train', '/kaggle/input/dacon-car-crash/train')
df

,sample_id,video_path,label
0,TRAIN_0000,/kaggle/input/dacon-car-crash/train/TRAIN_0000...,1
1,TRAIN_0001,/kaggle/input/dacon-car-crash/train/TRAIN_0001...,1
4,TRAIN_0004,/kaggle/input/dacon-car-crash/train/TRAIN_0004...,0
6,TRAIN_0006,/kaggle/input/dacon-car-crash/train/TRAIN_0006...,0
7,TRAIN_0007,/kaggle/input/dacon-car-crash/train/TRAIN_0007...,1
...,...,...,...
2685,TRAIN_2685,/kaggle/input/dacon-car-crash/train/TRAIN_2685...,1
2689,TRAIN_2689,/kaggle/input/dacon-car-crash/train/TRAIN_2689...,0
2692,TRAIN_2692,/kaggle/input/dacon-car-crash/train/TRAIN_2692...,1
2693,TRAIN_2693,/kaggle/input/dacon-car-crash/train/TRAIN_2693...,0


In [5]:
df.value_counts('label')

label
0    483
1    405
dtype: int64

In [6]:
# train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])
from sklearn.model_selection import KFold

kfold = KFold(n_splits=CFG['N_FOLDS'], shuffle=True, random_state=CFG['SEED'])
folds = list(kfold.split(df))

In [7]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list):
        self.video_path_list = video_path_list
        self.label_list = label_list
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
            img = img / 255.
            frames.append(img)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

In [8]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_score_min = np.Inf
 
    def __call__(self, val_score, model, model_path):
        score = val_score
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
            self.save_checkpoint(score, model, model_path)
 
    def save_checkpoint(self, val_score, model, model_path):
        if val_score < self.val_score_min:
            print(f'Validation score decreased ({self.val_score_min:.6f} --> {val_score:.6f}).  Saving model ...')
            torch.save(model.state_dict(), model_path)
        self.val_score_min = val_score
        
def train(model, optimizer, train_loader, val_loader, scheduler, device, fold):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    early_stop = EarlyStopping(patience=3, delta=0.001)
    
    train_loss_epoch, valid_loss_epoch, valid_score_epoch = [], [], []
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Fold {fold}, Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')

        train_loss_epoch.append(_train_loss)
        valid_loss_epoch.append(_val_loss)
        valid_score_epoch.append(_val_score)
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        early_stop(_val_score, model, f'ego_{fold}.pth')
        
        if early_stop.early_stop:
            print(f"Fold {fold}, Early stopping")
            break
    
    return model, train_loss_epoch, valid_loss_epoch, valid_score_epoch

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [9]:
train_loss_fold, valid_loss_fold, valid_score_fold = [], [], []

for fold in range(CFG['N_FOLDS']):
    print(f"{'='*20} Fold: {fold} {'='*20}")
    
    train_idx, val_idx = folds[fold]
    train_fold = df.iloc[train_idx].reset_index(drop=True)
    val_fold = df.iloc[val_idx].reset_index(drop=True)
    
    train_dataset = CustomDataset(train_fold['video_path'].values, train_fold['label'].values)
    train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
    
    val_dataset = CustomDataset(val_fold['video_path'].values, val_fold['label'].values)
    val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    model = models.video.r3d_18(pretrained=True)
    model.fc = nn.Linear(in_features=512, out_features=2)

    optimizer = torch.optim.AdamW(params=model.parameters(), lr=CFG["LEARNING_RATE"], weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
    
    _, train_loss_epoch, valid_loss_epoch, valid_score_epoch = train(model, optimizer, train_loader, val_loader, scheduler, device, fold)
    
    train_loss_fold.append(train_loss_epoch)
    valid_loss_fold.append(valid_loss_epoch)
    valid_score_fold.append(valid_score_epoch)


==================== Fold: 0 ====================


Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth


  0%|          | 0.00/127M [00:00<?, ?B/s]

  0%|          | 0/89 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

...

  0%|          | 0/23 [00:00<?, ?it/s]

Fold 4, Epoch [4], Train Loss : [0.08852] Val Loss : [0.55165] Val F1 : [0.83641]
EarlyStopping counter: 2 out of 3


  0%|          | 0/89 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

Fold 4, Epoch [5], Train Loss : [0.05274] Val Loss : [0.16791] Val F1 : [0.93124]
Epoch 00005: reducing learning rate of group 0 to 5.0000e-05.
EarlyStopping counter: 3 out of 3
Fold 4, Early stopping


In [10]:
preds_fold = []
test = pd.read_csv('/kaggle/input/dacon-car-crash/test.csv')
test['video_path'] = test['video_path'].str.replace('./test', '/kaggle/input/dacon-car-crash/test')

for fold in range(CFG['N_FOLDS']):
    model_path = f'ego_{fold}.pth'
    model = models.video.r3d_18(pretrained=False)
    model.fc = nn.Linear(in_features=512, out_features=2)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    
    test_dataset = CustomDataset(test['video_path'].values, None)
    test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

    preds = inference(model, test_loader, device)
    preds_fold.append(preds)

preds = np.array(preds_fold)
preds = np.where(np.sum(preds, axis=0)>len(preds)/2,1,0)
preds = preds.tolist()

submit = pd.read_csv('/kaggle/input/dacon-car-crash/sample_submission.csv')
submit['label'] = preds
submit.to_csv('./ego_fold.csv', index=False)

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

In [11]:
import requests

SLACK_API_TOKEN = 
CHANNEL_ID = 
message = "ego완료"
url = "https://slack.com/api/chat.postMessage"
headers = {
    "Authorization": f"Bearer {SLACK_API_TOKEN}",
    "Content-Type": "application/json"
}
payload = {
    "channel": CHANNEL_ID,
    "text": message
}
response = requests.post(url, headers=headers, json=payload)
if response.status_code == 200:
    print("Slack 메시지가 성공적으로 전송되었습니다.")
else:
    print("Slack 메시지 전송에 실패하였습니다.")
    print(response.text)

Slack 메시지가 성공적으로 전송되었습니다.
